In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#PART 1: Question Answering Task

In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 86.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6a668bbef4656c4e1af4534a803d49f39a3d72425a7af7eb991e63d7b9f69643
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [16]:
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments, BertTokenizer
from datasets import Dataset

In [5]:
# Read train data
with open(r"/kaggle/input/stanford-question-answering-dataset/train-v1.1.json", "r") as read_file:
    train = json.load(read_file)

In [6]:
# Read test data
with open(r"/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json", "r") as read_file:
    test = json.load(read_file)

In [8]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
config = {"max_length": 384}

In [9]:
# Preprocess function
def preprocess_function(question, context, answer_start_char, answer_end_char):
    inputs = tokenizer(
        question,
        context,
        max_length=config["max_length"],
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    offset = inputs.pop("offset_mapping")
    sequence_ids = inputs.sequence_ids()
    
    # Finding the start and end positions of the context in the tokenized input
    context_start = sequence_ids.index(1)
    context_end = len(sequence_ids) - sequence_ids[::-1].index(1)
    
    context_offsets = offset[context_start:context_end]
    
    # Create a mapping of character index to token index
    charcter_pos_to_token_pos = {}
    for token_pos, (char_start, char_end) in enumerate(context_offsets):
        for i in range(char_start, char_end):
            charcter_pos_to_token_pos[i] = token_pos + context_start
            
    start_pos = charcter_pos_to_token_pos.get(answer_start_char, 0)
    end_pos = charcter_pos_to_token_pos.get(
        answer_end_char - 1, 
        0 if start_pos == 0 else context_end - 1
    )
    
    inputs["start_positions"] = start_pos
    inputs["end_positions"] = end_pos
        
    return inputs

In [10]:
# Create dataset from JSON data
def create_dataset(data):
    input_ids_list = []
    attention_mask_list = []
    start_positions_list = []
    end_positions_list = []
    
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer_start_char = qa['answers'][0]['answer_start']
                answer_text = qa['answers'][0]['text']
                answer_end_char = answer_start_char + len(answer_text)
                
                inputs = preprocess_function(question, context, answer_start_char, answer_end_char)
                
                input_ids_list.append(inputs["input_ids"])
                attention_mask_list.append(inputs["attention_mask"])
                start_positions_list.append(inputs["start_positions"])
                end_positions_list.append(inputs["end_positions"])
    
    return Dataset.from_dict({
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "start_positions": start_positions_list,
        "end_positions": end_positions_list,
    })


In [11]:
train_dataset = create_dataset(train)
eval_dataset = create_dataset(test)

In [12]:
# Define token-level IoU custom metric
def token_level_iou(pred_start, pred_end, true_start, true_end):
    pred_range = set(range(pred_start, pred_end + 1))
    true_range = set(range(true_start, true_end + 1))
    
    intersection = len(pred_range & true_range)
    union = len(pred_range | true_range)
    
    return intersection / union if union != 0 else 0

In [13]:
train_dataset = train_dataset.select(range(1000))  # Train on a subset of 1000 examples
eval_dataset = eval_dataset.select(range(200))  

In [14]:
# Model initialization
model = BertForQuestionAnswering.from_pretrained("distilbert-base-uncased")


# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bi

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,4.372611
2,No log,4.165310
3,No log,4.258500


TrainOutput(global_step=375, training_loss=4.4952734375, metrics={'train_runtime': 6517.0887, 'train_samples_per_second': 0.46, 'train_steps_per_second': 0.058, 'total_flos': 587917702656000.0, 'train_loss': 4.4952734375, 'epoch': 3.0})

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [18]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [19]:
# Inference pipeline function
def inference_pipeline(question, context):
    # Encode the inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    
    # Get the model's predictions
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=token_type_ids)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # Debugging: Print the scores and input IDs
    print("Input IDs:", input_ids)
    print("Start Scores:", start_scores)
    print("End Scores:", end_scores)

    # Find the tokens with the highest start and end scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    # Debugging: Print the start and end positions
    print("Answer Start:", answer_start.item())
    print("Answer End:", answer_end.item())

    # Decode the tokens back to the answer text
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end])
    )

    return answer

In [20]:
# Example usage
question = "What is the capital of France?"
context = "Paris is the capital of France."
predicted_answer = inference_pipeline(question, context)
print(f"Predicted Answer: {predicted_answer}")

Input IDs: tensor([[ 101, 2054, 2003, 1996, 3007, 1997, 2605, 1029,  102, 3000, 2003, 1996,
         3007, 1997, 2605, 1012,  102]])
Start Scores: tensor([[-0.4313, -0.0596, -0.6571, -0.4706, -0.2744, -0.4489, -0.0829, -0.0429,
         -0.7337,  0.2652, -0.3306, -0.3272, -0.1265, -0.3221, -0.1276,  0.0280,
          0.0188]])
End Scores: tensor([[-0.3489, -0.0776,  0.5719,  0.4162,  0.4774,  0.1290,  0.2829,  0.5358,
         -0.2734,  0.4440,  0.5895,  0.3380,  0.5226,  0.0695,  0.4494,  0.2900,
          0.2908]])
Answer Start: 9
Answer End: 11
Predicted Answer: paris is
